In [1]:
# Import necessary libraries
import flet as ft
from flet.matplotlib_chart import MatplotlibChart
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import time
import random
import matplotlib.ticker as mticker
from matplotlib.ticker import FuncFormatter
import nest_asyncio
nest_asyncio.apply()

# Use SVG for better integration with Flet
matplotlib.use("svg")

In [2]:
# LBG Algorithm function
def lbg_algorithm(pdf_values, resolution, x_min, x_max, y_min, y_max, num_points, max_iterations, epsilon=1e-8, seed=0):
    
    """
    Performs the Linde-Buzo-Gray (LBG) algorithm for vector quantization.
    
    Parameters:
    pdf_values: The probability density function values.
    resolution: Grid resolution.
    x_min, x_max, y_min, y_max: Range of coordinates for the grid.
    num_points: Number of centroids (quantization points).
    max_iterations: Maximum number of iterations to perform.
    epsilon: Convergence threshold.
    seed: Random seed for reproducibility.
    
    Returns:
    centroids: Final centroids after the algorithm converges.
    distortions: Distortion values at each iteration.
    """
    
    # Generate grid of points
    x = np.linspace(x_min, x_max, resolution)
    y = np.linspace(y_min, y_max, resolution)
    xv, yv = np.meshgrid(x, y)
    grid = np.column_stack([xv.ravel(), yv.ravel()])

    # Initialize centroids randomly
    def initialize_centroids(grid, num_points, seed):
        np.random.seed(seed)  # Set seed for reproducibility
        indices = np.random.choice(len(grid), num_points, replace=False)
        return grid[indices]

    centroids = initialize_centroids(grid, num_points, seed)
    prev_distortion = float('inf')  # Initialize previous distortion to infinity
    distortions = []

    # Iterative process for LBG algorithm
    for iteration in range(max_iterations):
        
        # Assign each point to the closest centroid based on distance
        def assign_clusters_weighted(grid, pdf_values, centroids):
            distances = np.linalg.norm(grid[:, None] - centroids, axis=2)  # Euclidean distance
            cluster_indices = np.argmin(distances, axis=1)  # Assign points to nearest centroid
            return cluster_indices

        cluster_indices = assign_clusters_weighted(grid, pdf_values, centroids)

        # Update centroids as the weighted mean of assigned points, using the PDF as weights
        def update_centroids_weighted(grid, cluster_indices, pdf_values, num_points):
            new_centroids = []
            for m in range(num_points):
                cluster_points = grid[cluster_indices == m]  # Points assigned to centroid m
                cluster_weights = pdf_values[cluster_indices == m]  # Corresponding weights from the PDF
                if len(cluster_points) > 0:
                    # Calculate the weighted mean using the PDF as weights
                    weighted_mean = np.average(cluster_points, axis=0, weights=cluster_weights)
                    new_centroids.append(weighted_mean)
                else:
                    # If no points are assigned to this centroid, randomly select a point
                    new_centroids.append(np.random.choice(grid, size=1, axis=0)[0])
            return np.array(new_centroids)

        centroids = update_centroids_weighted(grid, cluster_indices, pdf_values, num_points)

        # Calculate the distortion (sum of weighted squared distances)
        def calculate_distortion(grid, pdf_values, centroids, cluster_indices):
            distortion = 0
            for i, centroid in enumerate(centroids):
                cluster_points = grid[cluster_indices == i]
                cluster_weights = pdf_values[cluster_indices == i]
                distortion += np.sum(cluster_weights * np.linalg.norm(cluster_points - centroid, axis=1)**2)
            return distortion / pdf_values.sum()

        distortion = calculate_distortion(grid, pdf_values, centroids, cluster_indices)
        distortions.append(distortion)

        # Convergence check: If change in distortion is small enough, stop
        if prev_distortion != float('inf') and np.isfinite(distortion):
            if np.abs(prev_distortion - distortion) / prev_distortion < epsilon:
                break
        prev_distortion = distortion

    return centroids, distortions

# Function to create distortion plot
def create_distortion_plot(distortions):
    """
    Creates a plot showing distortion vs iteration.

    Parameters:
    distortions: List of distortion values at each iteration.

    Returns:
    fig: Matplotlib figure object containing the plot.
    """
    fig, ax = plt.subplots(figsize=(5, 2.5))
    ax.plot(distortions)
    ax.set_title('Distortion vs Iteration')
    ax.set_xlabel('Iteration')
    ax.set_ylabel('Distortion')
    ax.yaxis.set_major_locator(mticker.MaxNLocator(integer=False))  # Allow non-integer ticks
    ax.yaxis.set_major_formatter(FuncFormatter(lambda x, pos: f'{x:.4f}'))  # Format y-axis to 4 decimals
    return fig

# Function to create quantization plot
def create_quantization_plot(grid, pdf_values, centroids):
    """
    Creates a plot showing the quantization points and the grid points.

    Parameters:
    grid: The grid of points.
    pdf_values: Probability density function values for the points.
    centroids: The optimized quantization points (centroids).

    Returns:
    fig: Matplotlib figure object containing the plot.
    """
    fig, ax = plt.subplots(figsize=(5, 2.5))
    ax.scatter(grid[:, 0], grid[:, 1], color='white', marker='.')
    ax.scatter(centroids[:, 0], centroids[:, 1], c='red', marker='x')
    ax.set_title('Optimized Quantization Points')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    return fig

# Main function to run the algorithm and update the page
def main(page: ft.Page):
    page.title = "Exercise 3 - Part B"
    page.window.width = 1050
    page.window.height = 650
    page.theme_mode = "light"

    results_container = ft.Column()

    # Function to execute the algorithm
    def run_algorithm(e=None):
        try:
            start_time = time.time()  # Start time for the run
            resolution = int(resolution_input.value)
            x_min, x_max, y_min, y_max = map(float, range_xy_input.value.split())
            num_points = int(num_points_input.value)
            iterations = int(iterations_input.value)
            seed = int(seed_input.value)  # Seed from user input
            pdf_code = pdf_input.value

            # Validate inputs
            if resolution <= 0 or num_points <= 0 or iterations <= 0:
                raise ValueError("Resolution, points, and iterations must be positive.")

            # Define and validate PDF function
            try:
                p = eval(f"lambda x, y: {pdf_code}")
                test_value = p(0, 0)
                if not isinstance(test_value, (int, float, np.ndarray)):
                    raise ValueError("PDF must return a numeric value.")
            except Exception as ex:
                raise ValueError(f"Error in PDF definition: {ex}")

            # Generate grid and PDF values
            x = np.linspace(x_min, x_max, resolution)
            y = np.linspace(y_min, y_max, resolution)
            xv, yv = np.meshgrid(x, y)
            grid = np.column_stack([xv.ravel(), yv.ravel()])
            pdf_values = np.array([p(x, y) for x, y in grid])
            pdf_values /= pdf_values.sum()  # Normalize the PDF

            # Run the LBG algorithm and find the best centroids with minimum distortion
            min_distortion = float('inf')
            best_centroids = None
            best_distortions = []

            for trial in range(iterations):
                dynamic_seed = seed + random.randint(0, 3000)  # Generate dynamic seed for each trial
                centroids, distortions = lbg_algorithm(pdf_values, resolution, x_min, x_max, y_min, y_max, num_points, iterations, seed=dynamic_seed)
                trial_distortion = distortions[-1]
                if trial_distortion < min_distortion:
                    min_distortion = trial_distortion
                    best_centroids = centroids
                    best_distortions = distortions

            # Create and display distortion and quantization plots
            distortion_fig = create_distortion_plot(best_distortions)
            quantization_fig = create_quantization_plot(grid, pdf_values, best_centroids)

            distortion_chart = MatplotlibChart(distortion_fig, expand=True)
            quantization_chart = MatplotlibChart(quantization_fig, expand=True)

            # Update UI with results
            results_container.controls.clear()
            results_container.controls.extend([distortion_chart, quantization_chart])
            Optimum_distortion.value = f"{min_distortion:.4f}"
            error_text.value = ""

        except Exception as ex:
            error_text.value = f"Error: {ex}"

        finally:
            # Record end time and update the runtime display
            error_text.update()
            page.update()
            end_time = time.time()
            elapsed_time = end_time - start_time
            run_time.value = f"{elapsed_time:.2f} seconds"
            run_time.update()

    # UI components (input fields, buttons, etc.)
    run_time = ft.Text(width=150, value="0.00 s", color="blue")
    pdf_input = ft.TextField(width=250, value="np.exp(-(x**2 + y**2) / 2)", multiline=True, on_blur=run_algorithm)
    resolution_input = ft.TextField(width=150, value="100", on_blur=run_algorithm)
    range_xy_input = ft.TextField(width=150, value="-2 2 -2 2", on_blur=run_algorithm)
    num_points_input = ft.TextField(width=150, value="7", on_blur=run_algorithm)
    iterations_input = ft.TextField(width=150, value="10", on_blur=run_algorithm)
    seed_input = ft.TextField(width=150, value="0", on_blur=run_algorithm)
    Optimum_distortion = ft.Text(width=150, value="0", color="blue")
    error_text = ft.Text(value="", color="red")
    exit_button = ft.ElevatedButton("EXIT", on_click=lambda e: page.window.close())
    
    # Layout setup for inputs and charts
    input_column = ft.Column(controls=[
        ft.Row([ft.Text('LBG Algorithm', weight=ft.FontWeight.BOLD, color="red")], alignment=ft.MainAxisAlignment.CENTER),
        ft.Row([ft.Text('Running time: '), run_time]),
        ft.Row([ft.Text('PDF to be normalized'), pdf_input]),
        ft.Row([ft.Text('Resolution'), resolution_input]),
        ft.Row([ft.Text('Range of (x,y)'), range_xy_input]),
        ft.Row([ft.Text('Num. of quantization points'), num_points_input]),
        ft.Row([ft.Text('Seed'), seed_input]),
        ft.Row([ft.Text('Global optimization iterations'), iterations_input]),
        ft.Row([ft.Text(f'Optimum distortion ({iterations_input.value} trials): '), Optimum_distortion]),
        ft.Row([exit_button], alignment=ft.MainAxisAlignment.CENTER),
    ], spacing=10)

    chart_column = ft.Column(controls=[results_container], expand=True)
    main_row = ft.Row(controls=[
        ft.Container(content=input_column, expand=3),
        ft.Container(content=chart_column, expand=4),
    ])
    page.add(main_row)
    page.add(error_text)
    
    # Initialize algorithm
    run_algorithm()



In [ ]:
# Run the app
# Works only on Windows 11 and macOS, If using windows 10 cooment this line and use the web browser option below
ft.app(target=main, view = ft.AppView.FLET_APP)

/var/folders/8m/rydr3dzn0rz4yd5k5v7_36980000gn/T/ipykernel_1556/2581806462.py:96: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(figsize=(5, 2.5))


In [ ]:
# Works om all operation systems, including Windows 10. Uncomment this line if using Windows 10 and comment the pop-up window option
#ft.app(target=main, view = ft.AppView.WEB_BROWSER)